In [20]:
from collections import namedtuple
from random import randint
import numpy
import numpy as np

BlockTransform = namedtuple('BlockTransform', ['x', 'y', 'mse'])

In [138]:
class Node:
    def __init__(self, t=None, x=0, y=0, size=4):
        self.childs = [None] * 4
        self.t = t
        self.x = x
        self.y = y
        self.size = size
        self.split = False

    def divide(self):
        self.split = True
        vals = {0: {'x': self.x, 'y': self.y, 'size': self.size // 2},
                1: {'x': self.x + self.size // 2, 'y': self.y, 'size': self.size // 2},
                2: {'x': self.x, 'y': self.y + self.size // 2, 'size': self.size // 2},
                3: {'x': self.x + self.size // 2, 'y': self.y + self.size // 2, 'size': self.size // 2},
                }
        for i in range(4):
            self.childs[i] = Node(**vals[i])

    def flatten(self, node = None):
        res = []
        node = node or self
        if node.split:
            res.append('split')
            for i in range(4):
                res += self.flatten(node.childs[i])
        else:
            res.append(node.t)
        return res
    def print(self, b = 0):
        node = self
        print('\t' * b + f"[{node.x} {node.y} {node.size}] = {b}")
        for node in self.childs:
            if node:
                node.print(b + 1)

class MaxHeap:
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self.size = 0
        self.Heap = [Node(t = BlockTransform(-1, -1, -1))] * (self.maxsize + 1)
        self.Heap[0] = Node(t = BlockTransform(-1, -1, 1000000000))
        self.FRONT = 1

    def parent(self, pos):
        return pos // 2

    def leftChild(self, pos):
        return 2 * pos

    def rightChild(self, pos):
        return (2 * pos) + 1

    def isLeaf(self, pos):
        if (self.size // 2) <= pos <= self.size:
            return True
        return False

    def swap(self, fpos, spos):
        self.Heap[fpos], self.Heap[spos] = (self.Heap[spos],
                                            self.Heap[fpos])
    def val(self, elem: Node):
        return elem.t.mse

    def maxHeapify(self, pos):
        if not self.isLeaf(pos):
            if (self.val(self.Heap[pos]) < self.val(self.Heap[self.leftChild(pos)]) or
                    self.val(self.Heap[pos]) < self.val(self.Heap[self.rightChild(pos)])):
                if (self.val(self.Heap[self.leftChild(pos)]) >
                        self.val(self.Heap[self.rightChild(pos)])):
                    self.swap(pos, self.leftChild(pos))
                    self.maxHeapify(self.leftChild(pos))
                else:
                    self.swap(pos, self.rightChild(pos))
                    self.maxHeapify(self.rightChild(pos))

    def insert(self, element: Node):
        if self.size >= self.maxsize:
            return
        self.size += 1
        self.Heap[self.size] = element
        current = self.size
        while (self.val(self.Heap[current]) >
               self.val(self.Heap[self.parent(current)])):
            self.swap(current, self.parent(current))
            current = self.parent(current)

    def print(self):
        for i in range(1, (self.size // 2) + 1):
            print(" PARENT : " + str(self.Heap[i].t) +
                  " LEFT CHILD : " + str(self.Heap[2 * i].t) +
                  " RIGHT CHILD : " + str(self.Heap[2 * i + 1].t))

    def pop(self):
        popped = self.Heap[self.FRONT]
        self.Heap[self.FRONT] = self.Heap[self.size]
        self.size -= 1
        self.maxHeapify(self.FRONT)
        return popped


def find_best_transform(*args):
    return BlockTransform(1, 2, randint(1, 10))


def compress(iters=100):
    image = np.triu(np.ones((4, 4)))
    rim = np.triu(np.ones((2, 2)))

    tree = Node()
    heap = MaxHeap(100)
    heap.insert(tree)
    for i in range(6):
        node = heap.pop()
        node.divide()
        for child in node.childs:
            child.data = find_best_transform(child.x, child.y, child.size)
            heap.insert(child)
    trans = tree.flatten()
    return trans

In [139]:
a = Node(t = BlockTransform(0, 0, 10))
b = MaxHeap(10)
b.insert(a)
a = Node(t = BlockTransform(0, 0, 20))
b.insert(a)
b.print()

 PARENT : BlockTransform(x=0, y=0, mse=20) LEFT CHILD : BlockTransform(x=0, y=0, mse=10) RIGHT CHILD : BlockTransform(x=-1, y=-1, mse=-1)
